# Machine Learning for Data Science - 2019-II
## Test 1 - Question 2 (corrected version)

* Student: Luis Vasquez Espinoza
* Code: 20152231J

____
**Q2:** Read the NetCDF file: sst2.cdf. Fill the missing temperatures by the
mean of the non-missing ones of the same day and month. Finally, serialize and
deserialize the new –ready to process– data structure. You can read the sst2.txt
file for a perspective of the previous file.

**Answer:**

In [31]:
# Libraries to be used
import pandas as pd
import xarray as xr

In [32]:
# Using defined tools

data_path_2 = 'data/sst2.cdf' # Relative path in my pc
sst2_df = xr.open_dataset(data_path_2)
sst2_df = sst2_df.to_dataframe()
sst2_df = sst2_df.sort_values(by='time')
sst2_df

T_20  QT_5020  ST_6020
depth lat  lon   time                                               
0.0   -2.0 250.0 2010-01-01 12:00:00  2.549000e+01      1.0      5.0
       0.0 265.0 2010-01-01 12:00:00  2.586000e+01      1.0      5.0
       2.0 250.0 2010-01-01 12:00:00  1.000000e+35      0.0      0.0
           265.0 2010-01-01 12:00:00  2.680000e+01      1.0      5.0
       0.0 250.0 2010-01-01 12:00:00  2.611000e+01      1.0      5.0
...                                            ...      ...      ...
           265.0 2018-12-31 12:00:00  1.000000e+35      0.0      0.0
      -2.0 265.0 2018-12-31 12:00:00  1.000000e+35      0.0      0.0
           250.0 2018-12-31 12:00:00  2.498616e+01      2.0      1.0
       2.0 250.0 2018-12-31 12:00:00  1.000000e+35      0.0      0.0
           265.0 2018-12-31 12:00:00  2.714600e+01      2.0      1.0

[19722 rows x 3 columns]

In [33]:
import numpy as np

sst2_df = sst2_df.reset_index()

# Using sst2.txt is expected to replace specific values for Nan
# Usually xarray does this automatically, but maybe library versions
# are conflicting

# The constant NAN_VALUE is defined to do this manually and
# start working from there

NAN_VALUE = 1.0000000409184788e+35

sst2_df = sst2_df.replace({'T_20': {NAN_VALUE: np.nan}})
sst2_df

,depth,lat,lon,time,T_20,QT_5020,ST_6020
0,0.0,-2.0,250.0,2010-01-01 12:00:00,25.490000,1.0,5.0
1,0.0,0.0,265.0,2010-01-01 12:00:00,25.860001,1.0,5.0
2,0.0,2.0,250.0,2010-01-01 12:00:00,NaN,0.0,0.0
3,0.0,2.0,265.0,2010-01-01 12:00:00,26.799999,1.0,5.0
4,0.0,0.0,250.0,2010-01-01 12:00:00,26.110001,1.0,5.0
...,...,...,...,...,...,...,...
19717,0.0,0.0,265.0,2018-12-31 12:00:00,NaN,0.0,0.0
19718,0.0,-2.0,265.0,2018-12-31 12:00:00,NaN,0.0,0.0
19719,0.0,-2.0,250.0,2018-12-31 12:00:00,24.986158,2.0,1.0
19720,0.0,2.0,250.0,2018-12-31 12:00:00,NaN,0.0,0.0


In [34]:
# Testing the process of separating temperatures data per buoy

temps_by_time = sst2_df.groupby('time')

new_features = ['T_20({})'.format(i) for i in range(6)] # For the six different buoys
new_features_data = [] # Storage list for later

for df in temps_by_time:
    buoys_temperature_data = list(df[1]['T_20']) # Getting temperature data of certain date
    new_features_data.append(buoys_temperature_data) # Adding it to main data list

In [35]:
# Storing temp data in dataframe
separeted_temps_df = pd.DataFrame(new_features_data, columns=new_features)
separeted_temps_df

,T_20(0),T_20(1),T_20(2),T_20(3),T_20(4),T_20(5)
0,25.490000,25.860001,NaN,26.799999,26.110001,25.230000
1,25.480000,26.820000,25.290001,25.920000,NaN,25.410000
2,NaN,25.520000,24.750000,26.850000,25.629999,25.250000
3,24.540001,25.600000,26.870001,25.580000,NaN,25.139999
4,24.670000,26.860001,25.580000,25.080000,25.600000,NaN
...,...,...,...,...,...,...
3282,26.427691,25.250563,NaN,NaN,24.793777,NaN
3283,25.243855,NaN,24.635138,NaN,NaN,26.587914
3284,NaN,24.656002,26.692562,25.181681,NaN,NaN
3285,NaN,24.538710,26.850328,25.119087,NaN,NaN


In [36]:
# Preparing times for new dataframe to consider per-bouys separated data

time_column = sst2_df['time'].unique()
unique_times_df = pd.DataFrame({'time':time_column})
unique_times_df

,time
0,2010-01-01 12:00:00
1,2010-01-02 12:00:00
2,2010-01-03 12:00:00
3,2010-01-04 12:00:00
4,2010-01-05 12:00:00
...,...
3282,2018-12-27 12:00:00
3283,2018-12-28 12:00:00
3284,2018-12-29 12:00:00
3285,2018-12-30 12:00:00


In [37]:
# Joining it with recently separated temperatures data

main_df = pd.concat([unique_times_df, separeted_temps_df], axis=1)
main_df

,time,T_20(0),T_20(1),T_20(2),T_20(3),T_20(4),T_20(5)
0,2010-01-01 12:00:00,25.490000,25.860001,NaN,26.799999,26.110001,25.230000
1,2010-01-02 12:00:00,25.480000,26.820000,25.290001,25.920000,NaN,25.410000
2,2010-01-03 12:00:00,NaN,25.520000,24.750000,26.850000,25.629999,25.250000
3,2010-01-04 12:00:00,24.540001,25.600000,26.870001,25.580000,NaN,25.139999
4,2010-01-05 12:00:00,24.670000,26.860001,25.580000,25.080000,25.600000,NaN
...,...,...,...,...,...,...,...
3282,2018-12-27 12:00:00,26.427691,25.250563,NaN,NaN,24.793777,NaN
3283,2018-12-28 12:00:00,25.243855,NaN,24.635138,NaN,NaN,26.587914
3284,2018-12-29 12:00:00,NaN,24.656002,26.692562,25.181681,NaN,NaN
3285,2018-12-30 12:00:00,NaN,24.538710,26.850328,25.119087,NaN,NaN


In [38]:
# Checking detected null temperatures
for temp_feature in new_features:
    print("Null temperatures in {}: {}".format(temp_feature, main_df[temp_feature].isna().sum()))

Null temperatures in T_20(0): 1205
Null temperatures in T_20(1): 1181
Null temperatures in T_20(2): 1184
Null temperatures in T_20(3): 1138
Null temperatures in T_20(4): 1183
Null temperatures in T_20(5): 1218


### In order to take use all temperatures of an specific day and month (for a single buoy), a method that retrives this data is needed.

In [39]:
# Adding day and month auxiliary features

main_df['day'] = main_df['time'].apply(lambda t: t.day)
main_df['month'] = main_df['time'].apply(lambda t: t.month)
main_df

,time,T_20(0),T_20(1),T_20(2),T_20(3),T_20(4),T_20(5),day,month
0,2010-01-01 12:00:00,25.490000,25.860001,NaN,26.799999,26.110001,25.230000,1,1
1,2010-01-02 12:00:00,25.480000,26.820000,25.290001,25.920000,NaN,25.410000,2,1
2,2010-01-03 12:00:00,NaN,25.520000,24.750000,26.850000,25.629999,25.250000,3,1
3,2010-01-04 12:00:00,24.540001,25.600000,26.870001,25.580000,NaN,25.139999,4,1
4,2010-01-05 12:00:00,24.670000,26.860001,25.580000,25.080000,25.600000,NaN,5,1
...,...,...,...,...,...,...,...,...,...
3282,2018-12-27 12:00:00,26.427691,25.250563,NaN,NaN,24.793777,NaN,27,12
3283,2018-12-28 12:00:00,25.243855,NaN,24.635138,NaN,NaN,26.587914,28,12
3284,2018-12-29 12:00:00,NaN,24.656002,26.692562,25.181681,NaN,NaN,29,12
3285,2018-12-30 12:00:00,NaN,24.538710,26.850328,25.119087,NaN,NaN,30,12


In [40]:
# Creating a indexing method
def get_data_by_date(dataframe, day, month):
    same_date_data = dataframe[(dataframe['day'] == day) & (dataframe['month'] == month)]
    return same_date_data

get_data_by_date(main_df, 3, 1) # All 3-January data

,time,T_20(0),T_20(1),T_20(2),T_20(3),T_20(4),T_20(5),day,month
2,2010-01-03 12:00:00,NaN,25.520000,24.750000,26.850000,25.629999,25.250000,3,1
367,2011-01-03 12:00:00,25.150000,22.700001,NaN,NaN,22.780001,22.190001,3,1
732,2012-01-03 12:00:00,23.020000,23.780001,NaN,25.780001,24.440001,22.980000,3,1
1098,2013-01-03 12:00:00,NaN,NaN,NaN,NaN,NaN,24.129999,3,1
1463,2014-01-03 12:00:00,NaN,25.463856,23.200001,NaN,23.590000,NaN,3,1
1828,2015-01-03 12:00:00,26.016737,24.967567,NaN,NaN,25.830313,24.128948,3,1
2193,2016-01-03 12:00:00,NaN,NaN,NaN,27.117937,NaN,NaN,3,1
2559,2017-01-03 12:00:00,22.525606,NaN,NaN,23.381256,22.747650,NaN,3,1
2924,2018-01-03 12:00:00,NaN,21.399014,21.714407,21.160917,22.101746,21.203608,3,1


In [41]:
# Remembering how the main dataframe looks right now
main_df

,time,T_20(0),T_20(1),T_20(2),T_20(3),T_20(4),T_20(5),day,month
0,2010-01-01 12:00:00,25.490000,25.860001,NaN,26.799999,26.110001,25.230000,1,1
1,2010-01-02 12:00:00,25.480000,26.820000,25.290001,25.920000,NaN,25.410000,2,1
2,2010-01-03 12:00:00,NaN,25.520000,24.750000,26.850000,25.629999,25.250000,3,1
3,2010-01-04 12:00:00,24.540001,25.600000,26.870001,25.580000,NaN,25.139999,4,1
4,2010-01-05 12:00:00,24.670000,26.860001,25.580000,25.080000,25.600000,NaN,5,1
...,...,...,...,...,...,...,...,...,...
3282,2018-12-27 12:00:00,26.427691,25.250563,NaN,NaN,24.793777,NaN,27,12
3283,2018-12-28 12:00:00,25.243855,NaN,24.635138,NaN,NaN,26.587914,28,12
3284,2018-12-29 12:00:00,NaN,24.656002,26.692562,25.181681,NaN,NaN,29,12
3285,2018-12-30 12:00:00,NaN,24.538710,26.850328,25.119087,NaN,NaN,30,12


In [42]:
# --- MAIN REPLACEMENT PROCESS ---

for temp_feature in new_features: # For each temperature column
    nan_indices = list( main_df[temp_feature].isna() ) # Getting data where buoy temp. is nan
    main_df.loc[nan_indices,temp_feature] = main_df[nan_indices].apply( # And in those places
        lambda d: get_data_by_date(main_df, d.day, d.month)[temp_feature].mean(), # Use method to get same day and month temperatures for buoy 'temp_feature', and get the mean
        axis=1) # And do this across rows

main_df

,time,T_20(0),T_20(1),T_20(2),T_20(3),T_20(4),T_20(5),day,month
0,2010-01-01 12:00:00,25.490000,25.860001,23.577296,26.799999,26.110001,25.230000,1,1
1,2010-01-02 12:00:00,25.480000,26.820000,25.290001,25.920000,22.541516,25.410000,2,1
2,2010-01-03 12:00:00,24.178086,25.520000,24.750000,26.850000,25.629999,25.250000,3,1
3,2010-01-04 12:00:00,24.540001,25.600000,26.870001,25.580000,23.245964,25.139999,4,1
4,2010-01-05 12:00:00,24.670000,26.860001,25.580000,25.080000,25.600000,24.683879,5,1
...,...,...,...,...,...,...,...,...,...
3282,2018-12-27 12:00:00,26.427691,25.250563,22.736074,23.898513,24.793777,23.193510,27,12
3283,2018-12-28 12:00:00,25.243855,23.093240,24.635138,24.309410,23.978578,26.587914,28,12
3284,2018-12-29 12:00:00,23.395435,24.656002,26.692562,25.181681,23.365755,22.914891,29,12
3285,2018-12-30 12:00:00,23.792972,24.538710,26.850328,25.119087,23.542344,23.611109,30,12


In [43]:
# Double-checking results
get_data_by_date(main_df, 3, 1) # New 3-January data, replaced with mean=24.178086 for this example

,time,T_20(0),T_20(1),T_20(2),T_20(3),T_20(4),T_20(5),day,month
2,2010-01-03 12:00:00,24.178086,25.520000,24.750000,26.850000,25.629999,25.250000,3,1
367,2011-01-03 12:00:00,25.150000,22.700001,23.221469,24.858022,22.780001,22.190001,3,1
732,2012-01-03 12:00:00,23.020000,23.780001,23.221469,25.780001,24.440001,22.980000,3,1
1098,2013-01-03 12:00:00,24.178086,23.971740,23.221469,24.858022,23.874244,24.129999,3,1
1463,2014-01-03 12:00:00,24.178086,25.463856,23.200001,24.858022,23.590000,23.313759,3,1
1828,2015-01-03 12:00:00,26.016737,24.967567,23.221469,24.858022,25.830313,24.128948,3,1
2193,2016-01-03 12:00:00,24.178086,23.971740,23.221469,27.117937,23.874244,23.313759,3,1
2559,2017-01-03 12:00:00,22.525606,23.971740,23.221469,23.381256,22.747650,23.313759,3,1
2924,2018-01-03 12:00:00,24.178086,21.399014,21.714407,21.160917,22.101746,21.203608,3,1


In [44]:
# Serializing dataframe
import pickle

saving_path = 'data/serial_clean_sst2.df' # Relative path in my pc

pickle.dump(main_df, open(saving_path, 'wb'))

In [45]:
# Loading the data to confirm

loaded_df2 = pickle.load(open(saving_path, 'rb'))

loaded_df2

,time,T_20(0),T_20(1),T_20(2),T_20(3),T_20(4),T_20(5),day,month
0,2010-01-01 12:00:00,25.490000,25.860001,23.577296,26.799999,26.110001,25.230000,1,1
1,2010-01-02 12:00:00,25.480000,26.820000,25.290001,25.920000,22.541516,25.410000,2,1
2,2010-01-03 12:00:00,24.178086,25.520000,24.750000,26.850000,25.629999,25.250000,3,1
3,2010-01-04 12:00:00,24.540001,25.600000,26.870001,25.580000,23.245964,25.139999,4,1
4,2010-01-05 12:00:00,24.670000,26.860001,25.580000,25.080000,25.600000,24.683879,5,1
...,...,...,...,...,...,...,...,...,...
3282,2018-12-27 12:00:00,26.427691,25.250563,22.736074,23.898513,24.793777,23.193510,27,12
3283,2018-12-28 12:00:00,25.243855,23.093240,24.635138,24.309410,23.978578,26.587914,28,12
3284,2018-12-29 12:00:00,23.395435,24.656002,26.692562,25.181681,23.365755,22.914891,29,12
3285,2018-12-30 12:00:00,23.792972,24.538710,26.850328,25.119087,23.542344,23.611109,30,12
